# Installations

In [ ]:
!pip install numpy
!pip install tensorflow-gpu
!pip install cudatoolkit
!pip install cudnn
!pip install livelossplot

# Set base configurations

In [ ]:
seed_value= 42

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)

import tensorflow as tf
tf.random.set_seed(seed_value)
tf.keras.utils.set_random_seed(seed_value)

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

# Import

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Conv2D, Flatten, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, BatchNormalization, Conv2DTranspose, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers, optimizers, losses
from livelossplot.inputs.tf_keras import PlotLossesCallback

# Initialization

In [ ]:
num_epochs  =
batch_size  =
cls_num     =
shape       =
lr          =
opt         = optimizers.Adamax(learning_rate=lr)
los         = losses.BinaryCrossentropy()
mtr         = []

In [ ]:
address         =
address_train   =
address_test    =
address_val     =

data_train      = os.listdir(address_train)
data_test       = os.listdir(address_test)
data_val        = os.listdir(address_val)

steps_per_train =
steps_per_test  =
steps_per_val   =

# Data preparation

# Functions

# Data generator

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, address, data, shape, batch_size, cls_num, shuffle=True):
        self.address    = address
        self.data       = data
        self.shape      = shape
        self.batch_size = batch_size
        self.cls_num    = cls_num
        self.shuffle    = shuffle

        self.on_epoch_end()


    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))


    def __getitem__(self, index):
        indexes       = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.data[k] for k in indexes]
        x, y          = self.__data_generation(list_IDs_temp)
        return x, y


    def on_epoch_end(self):
        self.indexes = np.arange(len(self.data))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)


    def __data_generation(self, list_IDs_temp):
        x = np.empty((self.batch_size, int(self.shape[0]), int(self.shape[1]), int(self.shape[2])))
        y = np.empty((self.batch_size), dtype=int)

        for i, ID in enumerate(list_IDs_temp):
            x[i,] =
            y[i]  =

        return x, to_categorical(y, num_classes=self.cls_num)


train_gen = DataGenerator(address_train, data_train, shape, batch_size, cls_num)
test_gen  = DataGenerator(address_test , data_test , shape, batch_size, cls_num)
val_gen   = DataGenerator(address_val  , data_val  , shape, batch_size, cls_num)

# Model

In [ ]:
def get_model(myshape, cls_num):

  ##############################################################################

  pad =
  act =
  ect =
  drp =

  ##############################################################################

  inputt = Input(shape=myshape)

  ##############################################################################

  x = Flatten()(y)

  outputt = Dense(cls_num, activation=ect)(x)

  return Model(inputs=inputt, outputs=outputt)

model = get_model()
model.compile(optimizer=opt, loss=los, metrics=mtr)
model.summary()

# Train

In [ ]:
def callback():
  mymonitor = 'val_loss'
  mymode    = 'min'

  main_chk  = ModelCheckpoint(filepath='my_checkpoint', monitor=mymonitor, mode=,mymode verbose=1, save_best_only=True)
  early_st  = EarlyStopping(monitor=mymonitor, mode=mymode, patience=30, verbose=1)
  rduce_lr  = ReduceLROnPlateau(monitor=mymonitor, mode=mymode, factor=0.5, patience=5, verbose=1, min_lr=0.0001)
  tr_plot   = TrainingPlot()

  return [main_chk, early_st, rduce_lr, tr_plot]

history = model.fit(train_gen,
                    validation_data=val_gen,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=1,
                    callbacks=callback(),
                    steps_per_epoch  = steps_per_train,
                    validation_steps = steps_per_val)

# Evaluation

In [ ]:
testmodel = load_model('my_checkpoint', compile=True)
tst_loss , tst_acc = testmodel.evaluate(test_gen, steps = steps_per_test)